In [17]:
# Importar bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
import os

# Criar pasta para salvar gráficos
output_dir = "figuras"
os.makedirs(output_dir, exist_ok=True)

# Parâmetros do problema
k = 1.0         # Constante elástica (N/m)
m_val = 1.0     # Massa padrão (kg)
m_defeito = 5.0 * m_val  # Massa com defeito (5x maior)

# Função para construir a matriz dinâmica do sistema
def construir_matriz_dinamica(N, defeito=False):
    M = np.ones(N) * m_val
    if defeito:
        M[N // 2] = m_defeito  # Introduzir defeito de massa no centro

    D = np.zeros((N, N))
    for i in range(N):
        D[i, i] = 2 * k / M[i]
        D[i, (i - 1) % N] = -k / M[i]
        D[i, (i + 1) % N] = -k / M[i]
    return D

# Função para calcular as frequências naturais e modos normais
def calcular_frequencias_modos(D):
    w2, modos = eigh(D)               # w2 são os autovalores
    w = np.sqrt(np.abs(w2))           # Frequências naturais
    return w, modos

# Função para gerar e salvar o histograma das frequências
def plotar_histograma_frequencias(w, N, com_defeito):
    plt.figure(figsize=(8, 4))
    plt.hist(w, bins=7, density=True, alpha=0.7, color='steelblue')
    titulo = f"Densidade de Estados - N={N} " + ("(com defeito)" if com_defeito else "(homogêneo)")
    plt.title(titulo)
    plt.xlabel("Frequência (rad/s)")
    plt.ylabel("Densidade")
    plt.grid(True)
    plt.tight_layout()
    filename = f"histograma_N{N}_{'defeito' if com_defeito else 'homogeneo'}.png"
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()
    return filename

# Função para visualizar os modos normais (deslocamentos relativos)
def plotar_modos_normais(modos, N, w, tipo, qtd=5):
    indices = np.argsort(w)
    if tipo == "baixas":
        indices = indices[:qtd]
    else:
        indices = indices[-qtd:]

    plt.figure(figsize=(10, 6))
    for i in indices:
        plt.plot(modos[:, i], label=f'Modo {i} - f={w[i]:.2f}')
    plt.title(f'Modos de {"Baixa" if tipo == "baixas" else "Alta"} Frequência - N={N}')
    plt.xlabel("Índice da Massa")
    plt.ylabel("Deslocamento Relativo")
    plt.legend()
    plt.tight_layout()
    filename = f"modos_{tipo}_N{N}.png"
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()
    return filename

# Executar simulações para N=100 (exemplo menor e visual)
resultados = {}

N = 1000
for defeito in [False, True]:
    D = construir_matriz_dinamica(N, defeito=defeito)
    w, modos = calcular_frequencias_modos(D)
    hist_fig = plotar_histograma_frequencias(w, N, com_defeito=defeito)
    
    modos_baixos = plotar_modos_normais(modos, N, w, "baixas")
    modos_altos = plotar_modos_normais(modos, N, w, "altas")

    resultados[f"N={N}, defeito={defeito}"] = {
        "frequencias": w,
        "modos": modos,
        "histograma": hist_fig,
        "modos_baixos": modos_baixos,
        "modos_altos": modos_altos
    }

print("Simulações finalizadas e gráficos salvos na pasta 'figuras'.")


Simulações finalizadas e gráficos salvos na pasta 'figuras'.


In [14]:
D = construir_matriz_dinamica(1000, defeito=defeito)
D

array([[ 2., -1.,  0., ...,  0.,  0., -1.],
       [-1.,  2., -1., ...,  0.,  0.,  0.],
       [ 0., -1.,  2., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  2., -1.,  0.],
       [ 0.,  0.,  0., ..., -1.,  2., -1.],
       [-1.,  0.,  0., ...,  0., -1.,  2.]])

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import eigh
from pathlib import Path

# Configurar estilo seaborn
sns.set(style="whitegrid")

# Criar pasta para salvar gráficos
output_dir = Path("figuras")
output_dir.mkdir(exist_ok=True)

# Parâmetros do problema
k = 1.0
m_val = 1.0
m_defeito = 5.0 * m_val

def construir_matriz_dinamica(N, defeito=False):
    M = np.ones(N) * m_val
    if defeito:
        M[N // 2] = m_defeito

    D = np.zeros((N, N))
    for i in range(N):
        D[i, i] = 2 * k / M[i]
        D[i, (i - 1) % N] = -k / M[i]
        D[i, (i + 1) % N] = -k / M[i]
    return D

def calcular_frequencias_modos(D):
    w2, modos = eigh(D)
    w = np.sqrt(np.abs(w2))
    return w, modos

def plotar_histograma_frequencias(w, N, com_defeito, bins=50):
    plt.figure(figsize=(10,5))
    sns.histplot(w, bins=bins, kde=True, color='steelblue', stat='density')
    titulo = f"Densidade de Estados - N={N} " + ("(com defeito)" if com_defeito else "(homogêneo)")
    plt.title(titulo)
    plt.xlabel("Frequência (rad/s)")
    plt.ylabel("Densidade")
    plt.tight_layout()
    filename = output_dir / f"histograma_N{N}_{'defeito' if com_defeito else 'homogeneo'}.png"
    plt.savefig(filename)
    plt.close()
    return str(filename)

def plotar_modos_normais(modos, N, w, tipo, qtd=5):
    indices = np.argsort(w)
    if tipo == "baixas":
        indices = indices[:qtd]
    else:
        indices = indices[-qtd:]

    plt.figure(figsize=(10,6))
    for i in indices:
        plt.plot(modos[:, i], label=f'Modo {i} - f={w[i]:.2f}')
    plt.title(f'Modos de {"Baixa" if tipo=="baixas" else "Alta"} Frequência - N={N}')
    plt.xlabel("Índice da Massa")
    plt.ylabel("Deslocamento Relativo")
    plt.legend()
    plt.tight_layout()
    filename = output_dir / f"modos_{tipo}_N{N}.png"
    plt.savefig(filename)
    plt.close()
    return str(filename)

# Execução principal
resultados = {}
N = 1000

for defeito in [False, True]:
    D = construir_matriz_dinamica(N, defeito=defeito)
    w, modos = calcular_frequencias_modos(D)

    hist_fig = plotar_histograma_frequencias(w, N, com_defeito=defeito)
    modos_baixos = plotar_modos_normais(modos, N, w, "baixas")
    modos_altos = plotar_modos_normais(modos, N, w, "altas")

    resultados[f"N={N}, defeito={defeito}"] = {
        "frequencias": w,
        "modos": modos,
        "histograma": hist_fig,
        "modos_baixos": modos_baixos,
        "modos_altos": modos_altos
    }

print("Simulações finalizadas e gráficos salvos na pasta 'figuras'.")


Simulações finalizadas e gráficos salvos na pasta 'figuras'.


In [18]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import eigh
from pathlib import Path

# Configurar estilo seaborn
sns.set(style="whitegrid")

# Criar pasta para salvar gráficos
output_dir = Path("figuras")
output_dir.mkdir(exist_ok=True)

# Parâmetros do problema
k = 1.0
m_val = 1.0
m_defeito = 5.0 * m_val

def construir_matriz_dinamica(N, defeito=False):
    M = np.ones(N) * m_val
    if defeito:
        M[N // 2] = m_defeito

    D = np.zeros((N, N))
    for i in range(N):
        D[i, i] = 2 * k / M[i]
        D[i, (i - 1) % N] = -k / M[i]
        D[i, (i + 1) % N] = -k / M[i]
    return D

def calcular_frequencias_modos(D):
    w2, modos = eigh(D)
    w = np.sqrt(np.abs(w2))
    return w, modos

def plotar_histograma_frequencias(w, N, com_defeito, bins=50):
    plt.figure(figsize=(10,5))
    sns.histplot(w, bins=bins, kde=True, color='steelblue', stat='density')
    titulo = f"Densidade de Estados - N={N} " + ("(com defeito)" if com_defeito else "(homogêneo)")
    plt.title(titulo)
    plt.xlabel("Frequência (rad/s)")
    plt.ylabel("Densidade")
    plt.tight_layout()
    filename = output_dir / f"histograma_N{N}_{'defeito' if com_defeito else 'homogeneo'}.png"
    plt.savefig(filename)
    plt.close()
    return str(filename)

def plotar_modos_normais(modos, N, w, tipo, qtd=5):
    indices = np.argsort(w)
    if tipo == "baixas":
        indices = indices[:qtd]
    else:
        indices = indices[-qtd:]

    plt.figure(figsize=(10,6))
    for i in indices:
        plt.plot(modos[:, i], label=f'Modo {i} - f={w[i]:.2f}')
    plt.title(f'Modos de {"Baixa" if tipo=="baixas" else "Alta"} Frequência - N={N}')
    plt.xlabel("Índice da Massa")
    plt.ylabel("Deslocamento Relativo")
    plt.legend()
    plt.tight_layout()
    filename = output_dir / f"modos_{tipo}_N{N}.png"
    plt.savefig(filename)
    plt.close()
    return str(filename)

# Execução principal
resultados = {}
N = 1000

for defeito in [False, True]:
    D = construir_matriz_dinamica(N, defeito=defeito)
    w, modos = calcular_frequencias_modos(D)

    hist_fig = plotar_histograma_frequencias(w, N, com_defeito=defeito)
    modos_baixos = plotar_modos_normais(modos, N, w, "baixas")
    modos_altos = plotar_modos_normais(modos, N, w, "altas")

    resultados[f"N={N}, defeito={defeito}"] = {
        "frequencias": w,
        "modos": modos,
        "histograma": hist_fig,
        "modos_baixos": modos_baixos,
        "modos_altos": modos_altos
    }

print("Simulações finalizadas e gráficos salvos na pasta 'figuras'.")


Simulações finalizadas e gráficos salvos na pasta 'figuras'.


In [20]:
import numpy as np
from scipy.linalg import eigh
from bokeh.plotting import figure, output_file, save
from bokeh.layouts import gridplot
from pathlib import Path

# Criar pasta para salvar gráficos
output_dir = Path("figuras")
output_dir.mkdir(exist_ok=True)

k = 1.0
m_val = 1.0
m_defeito = 5.0 * m_val

def construir_matriz_dinamica(N, defeito=False):
    M = np.ones(N) * m_val
    if defeito:
        M[N // 2] = m_defeito

    D = np.zeros((N, N))
    for i in range(N):
        D[i, i] = 2 * k / M[i]
        D[i, (i - 1) % N] = -k / M[i]
        D[i, (i + 1) % N] = -k / M[i]
    return D

def calcular_frequencias_modos(D):
    w2, modos = eigh(D)
    w = np.sqrt(np.abs(w2))
    return w, modos

def plotar_histograma_bokeh(w, N, com_defeito, bins=50):
    hist, edges = np.histogram(w, bins=bins, density=True)
    p = figure(title=f"Densidade de Estados - N={N} {'(com defeito)' if com_defeito else '(homogêneo)'}",
               x_axis_label='Frequência (rad/s)', y_axis_label='Densidade', width=700, height=400)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", alpha=0.5)
    filename = output_dir / f"histograma_N{N}_{'defeito' if com_defeito else 'homogeneo'}.html"
    output_file(str(filename))
    save(p)
    return str(filename)

def plotar_modos_bokeh(modos, N, w, tipo, qtd=5):
    indices = np.argsort(w)
    if tipo == "baixas":
        indices = indices[:qtd]
    else:
        indices = indices[-qtd:]
    
    p = figure(title=f'Modos de {"Baixa" if tipo=="baixas" else "Alta"} Frequência - N={N}',
               x_axis_label='Índice da Massa', y_axis_label='Deslocamento Relativo', width=700, height=400)
    
    for i in indices:
        p.line(x=np.arange(N), y=modos[:, i], legend_label=f'Modo {i} - f={w[i]:.2f}')
    
    p.legend.location = "top_right"
    filename = output_dir / f"modos_{tipo}_N{N}.html"
    output_file(str(filename))
    save(p)
    return str(filename)

# Execução principal
resultados = {}
N = 1000

for defeito in [False, True]:
    D = construir_matriz_dinamica(N, defeito=defeito)
    w, modos = calcular_frequencias_modos(D)

    hist_file = plotar_histograma_bokeh(w, N, com_defeito=defeito)
    modos_baixos_file = plotar_modos_bokeh(modos, N, w, "baixas")
    modos_altos_file = plotar_modos_bokeh(modos, N, w, "altas")

    resultados[f"N={N}, defeito={defeito}"] = {
        "frequencias": w,
        "modos": modos,
        "histograma": hist_file,
        "modos_baixos": modos_baixos_file,
        "modos_altos": modos_altos_file
    }

print("Simulações finalizadas e gráficos salvos na pasta 'figuras'.")


Simulações finalizadas e gráficos salvos na pasta 'figuras'.
